In [266]:
import csv
import os
import glob
import zarr
import tifffile
import numpy as np
import dask.array as da
from lxml import etree
import napari

%gui qt

In [251]:
def convert_u16_to_u8(im_u16):
    """Converts a uint16 image into a uint8 image after scaling. 
    This function matches scripts used to produce uint8 .tif files from LIMS jp2 images. 
    """
    assert im_u16.dtype == 'uint16', "Input must be uint16"
    return np.uint8(np.round(im_u16.astype(float)*(2**8)/(2**16-1)))

In [386]:
slice_n = '0070'
csv_path = '/Users/elyse.morin/OneDrive - Allen Institute/Transgenic_Technology_Elyse/Projects/Unet_test_output/547656/csv/'
zarr_img_path = '/Users/elyse.morin/OneDrive - Allen Institute/Transgenic_Technology_Elyse/Projects/Unet_test_output/547656/zarr/'
tif_img_crop_path = '/Users/elyse.morin/OneDrive - Allen Institute/Transgenic_Technology_Elyse/Projects/Unet_test_output/547656/tif_crop/'


#get barcode/zarr file prefix
files = glob.glob(str(zarr_img_path+'*.zarr'))
head, tail = os.path.split(files[0])
barcode = tail.split('-')[0]

#read coordinates and image
g_coord = np.loadtxt(str(csv_path+'/'+barcode+'-'+slice_n+'_g.csv'), skiprows=1, delimiter=',')[:,:2]
r_coord = np.loadtxt(str(csv_path+'/'+barcode+'-'+slice_n+'_r.csv'), skiprows=1, delimiter=',')[:,:2]

image = da.from_zarr(str(zarr_img_path+'/'+barcode+'-'+slice_n+'.zarr'))

In [440]:
#crop image
nrow = 3
ncolumn = 5

image_crop_u16 = image[nrow*2048:(nrow+1)*2048,ncolumn*2048:(ncolumn+1)*2048,:]
image_crop_u8 = convert_u16_to_u8(image_crop_u16)

tifffile.imwrite(
    tif_img_crop_path+barcode+'-'+slice_n+'_val_anno_'+str(nrow)+'_'+str(ncolumn)+'.tif',
    image_crop_u8)

#crop coordinates
g_coord_crop = g_coord[(g_coord[:, 0] >= nrow*2048) & (g_coord[:, 0] < (nrow+1)*2048) & (g_coord[:, 1] >= ncolumn*2048) & (g_coord[:, 1] < (ncolumn+1)*2048)]
r_coord_crop = r_coord[(r_coord[:, 0] >= nrow*2048) & (r_coord[:, 0] < (nrow+1)*2048) & (r_coord[:, 1] >= ncolumn*2048) & (r_coord[:, 1] < (ncolumn+1)*2048)]

g_coord_crop[:, 0] -= nrow*2048
g_coord_crop[:, 1] -= ncolumn*2048
r_coord_crop[:, 0] -= nrow*2048
r_coord_crop[:, 1] -= ncolumn*2048

g_coord_crop_flipxy = np.flip(g_coord_crop, axis=1)
r_coord_crop_flipxy = np.flip(r_coord_crop, axis=1)

## Axes need to be flipped to be read into ImageJ correctly
np.savetxt(tif_img_crop_path+barcode+'-'+slice_n+'_val_anno_'+str(nrow)+'_'+str(ncolumn)+'_g.csv', np.round(g_coord_crop_flipxy), fmt='%i', delimiter=",", header="x,y", comments="")
np.savetxt(tif_img_crop_path+barcode+'-'+slice_n+'_val_anno_'+str(nrow)+'_'+str(ncolumn)+'_r.csv', np.round(r_coord_crop_flipxy), fmt='%i', delimiter=",", header="x,y", comments="")

In [441]:
#csv to xml

root=etree.Element('CellCounter_Marker_File')
tree=etree.ElementTree(root)
image_prop = etree.Element('Image_Properties')
image_fname = etree.Element('Image_Filename')
root.append(image_prop)
image_prop.append(image_fname)
image_fname.text=str(barcode+'-'+slice_n+'_val_anno_'+str(nrow)+'_'+str(ncolumn)+'.tif')

marker_data = etree.Element('Marker_Data')
root.append(marker_data)
current_type1 = etree.Element('Current_Type')
marker_data.append(current_type1)
current_type1.text='1'
marker_type1 = etree.Element('Marker_Type')
marker_data.append(marker_type1)
type1 = etree.Element('Type')
type1.text = '1'
marker_type1.append(type1)

with open(tif_img_crop_path+barcode+'-'+slice_n+'_val_anno_'+str(nrow)+'_'+str(ncolumn)+'_g.csv') as f:
    reader = csv.DictReader(f)
    for row in reader:
        row['MarkerX'] = row['x']
        row['MarkerY'] = row['y']
        del row['x']
        del row['y']
        row['MarkerZ'] = 1
        surface_elem = etree.SubElement(marker_type1, "Marker")
        for elem_name, elem_value in row.items():
            etree.SubElement(surface_elem, elem_name.strip()).text = str(elem_value)

marker_type2 = etree.Element('Marker_Type')
marker_data.append(marker_type2)
type2 = etree.Element('Type')
type2.text = '2'
marker_type2.append(type2)

with open(tif_img_crop_path+barcode+'-'+slice_n+'_val_anno_'+str(nrow)+'_'+str(ncolumn)+'_r.csv') as f:
    reader = csv.DictReader(f)
    for row in reader:
        row['MarkerX'] = row['x']
        row['MarkerY'] = row['y']
        del row['x']
        del row['y']
        row['MarkerZ'] = 1
        surface_elem = etree.SubElement(marker_type2, "Marker")
        for elem_name, elem_value in row.items():
            etree.SubElement(surface_elem, elem_name.strip()).text = str(elem_value)

tree.write(open(tif_img_crop_path+barcode+'-'+slice_n+'_val_anno_'+str(nrow)+'_'+str(ncolumn)+'.xml','wb'), pretty_print=True,encoding='UTF-8',xml_declaration=True)

In [442]:
#view composite image with red and green labels
viewer = napari.view_image(image_crop_u8, name='slice')
viewer.add_points(g_coord_crop, size=5, face_color='lime',edge_color='lime')
viewer.add_points(r_coord_crop, size=5, face_color='red',edge_color='red')

<Points layer 'r_coord_crop' at 0x17f545c40>

In [382]:
#view composite image with red and green labels
viewer = napari.view_image(image, name='slice')
viewer.add_points(g_coord, size=5, face_color='lime',edge_color='lime')
viewer.add_points(r_coord, size=5, face_color='red',edge_color='red')

<Points layer 'r_coord' at 0x16831d730>

In [296]:
#view green channel with labels
viewer = napari.view_image(image[:,:,1], name='green channel')
viewer.add_points(g_coord, size=5, face_color='lime',edge_color='lime')

<Points layer 'g_coord' at 0x15e7df3d0>